In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv
import pandas as pd
import os

load_dotenv()

True

In [30]:
books = pd.read_csv("Dataset/7k books/books_cleaned.csv")
books.shape

(4493, 13)

In [31]:
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."


In [32]:
books['tagged_description'].to_csv("Dataset/7k books/tagged_description.txt", sep='\n', index=False, header=False)

In [33]:
loader = TextLoader("Dataset/7k books/tagged_description.txt", encoding='utf-8')
raw_documents = loader.load()
text_splitter = CharacterTextSplitter(separator='\n', chunk_size=0, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

In [37]:
documents[0]

Document(metadata={'source': 'Dataset/7k books/tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of l

In [51]:
api_key = os.getenv("HUGGINGFACEHUB_API_KEY")
embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db_books = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory="chroma_db"
)

In [55]:
query = "A book about time travel"
docs = db_books.similarity_search(query, k=10)
docs

[Document(id='d94068a1-739f-4f62-80d5-084f0f9e1455', metadata={'source': 'Dataset/7k books/tagged_description.txt'}, page_content='9780451528551 A classic novel of the future follows the Time Traveller as he hurtles one million years into the future and encounters a world populated by two distinct races, the childlike Eloi and the disgusting Morlocks who prey on the Eloi. Reprint.'),
 Document(id='9b2e16ac-a37f-4088-b5e7-92bd950faec3', metadata={'source': 'Dataset/7k books/tagged_description.txt'}, page_content='"9780553380163 An anniversary edition of a now-classic survey of the origin and nature of the universe features a new introduction by the author and a new chapter on the possibility of time travel and ""wormholes"" in space"'),
 Document(id='bf9d0c7d-39f9-4a91-af59-b16eb979edb4', metadata={'source': 'Dataset/7k books/tagged_description.txt'}, page_content="9780141005348 Is time travel really possible? Can we break the last cosmic taboo? Yes, says internationally acclaimed write